# Raster and vector data
- ECE471 - Fundamentals of Remote Sensing and Earth Observation
- Summer IS 2022
- Krishna Karra

## Calf canyon fire

### Grab Sentinel-2 imagery from GEE

In [ ]:
import ee
ee.Initialize()

In [ ]:
# define a point over Santa Fe
fc = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {},
            "geometry": {
                "type": "Point",
                "coordinates": [-105.84640502929686, 35.78885488168885],
            },
        }
    ],
}

In [ ]:
# define some image parameters
start_datetime = '2022-05-08'
end_datetime = '2022-05-10'
collection_id = 'COPERNICUS/S2_SR'
bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']

In [ ]:
# build ROI
roi = ee.Geometry(fc['features'][0]['geometry'])

In [ ]:
# build image collection
ic = ee.ImageCollection(collection_id).filterDate(start_datetime, end_datetime).filterBounds(roi)

In [ ]:
# grab image from image collection
img = ic.first().select(bands)

In [ ]:
# export to google drive
task = ee.batch.Export.image.toDrive(img, description='S2_SantaFe_example', maxPixels=1e12)
task.start()

### Inspect the data with GDAL

In [ ]:
!gdalinfo S2_SantaFe_example.tif

### Read in and display

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import rasterio
%matplotlib inline
from pprint import pprint

In [ ]:
raster_file = 'S2_SantaFe_example.tif'
src = rasterio.open(raster_file, 'r')

In [ ]:
pprint(src.profile)

In [ ]:
data = src.read()

In [ ]:
# plot RGB
rgb = np.stack((data[2, :, :], data[1, :, :], data[0, :, :]), axis=-1)
plt.figure(dpi=300)
plt.imshow(rgb/4000.0)
plt.axis('off')
plt.title('RGB Santa Fe area')

## Grab fire perimeters from USFS
https://ftp.wildfire.gov/public/incident_specific_data/southwest/GACC_Incidents/2022/2022_Hermits_Peak/IR/

In [ ]:
import geopandas as gpd

In [ ]:
perimeter_file = 'fire_perimeter.json'
gdf = gpd.read_file(perimeter_file)

In [ ]:
gdf.plot()

### Clip the raster image to the fire boundary

In [ ]:
!gdalwarp -co COMPRESS=LZW -co BIGTIFF=YES -cutline fire_perimeter.json -crop_to_cutline S2_SantaFe_example.tif S2_crop.tif

In [ ]:
!gdalinfo S2_crop.tif